In [1]:
from selenium import webdriver
from selenium.common import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager  # To download Edge WebDriver
from selenium.common.exceptions import NoSuchElementException
from itertools import chain
import pandas as pd
import os
import time
import numpy as np

In [2]:
#cấu hình trình duyệt web
edge_options = Options()
edge_options.add_argument('--disable-blink-features=AutomationControlled')
edge_options.add_argument("--disable-extensions")
edge_options.add_argument("--no-sandbox")
# edge_options.add_argument("--headless")

In [3]:
# Khởi tạo WebDriver
service = Service(EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service, options=edge_options)

In [17]:
def getURL(ori_url):
    # Đường dẫn đến msedgedriver
    driver_path = "C:/WebDriver/msedgedriver.exe"  # Thay đổi đường dẫn theo nơi bạn đặt msedgedriver
    
    # Cấu hình dịch vụ cho Microsoft Edge
    service = Service(executable_path=driver_path)
    
    # Khởi tạo driver cho Microsoft Edge
    driver = webdriver.Edge(service=service)
    
    # Danh sách để lưu các đường link
    download_links = []
    
    try:
        # Mở đường dẫn
        driver.get(ori_url)
        time.sleep(2)  # Đợi trang tải xong

        # Tìm phần tử cần click bằng XPath (các bước click trước đó của bạn)
        element_to_click1 = driver.find_element(By.XPATH, '//*[@id="pagewrap"]/div[1]/div[3]/div[1]/div[1]/div[2]/div[2]/div/label/i')
        element_to_click1.click()
        time.sleep(2)
        
        element_to_click2 = driver.find_element(By.XPATH, '//*[@id="pagewrap"]/div[1]/div[3]/div[1]/div[1]/div[2]/div[2]/div/div/div[5]')
        element_to_click2.click()
        
        element_to_click2 = driver.find_element(By.XPATH, '//*[@id="pagewrap"]/div[1]/div[3]/div[1]/div[1]/div[2]/div[2]/div/div/div[6]')
        element_to_click2.click()

        # Đợi một chút để trang tải hoàn toàn sau khi click
        time.sleep(5)

        # Tìm tất cả các phần tử Download trong cột "Tài về"
        download_elements = driver.find_elements(By.XPATH, '//table//tr/td[last()]/a')  # Cột cuối cùng chứa biểu tượng Download
        
        # Lấy thuộc tính href từ từng phần tử và lọc các link
        for element in download_elements:
            link = element.get_attribute('href')
            if link:  # Kiểm tra nếu link không rỗng
                # Chỉ thêm link nếu nó chứa đuôi .pdf hoặc chứa đường dẫn của file báo cáo
                if link.endswith('.pdf') or '/BCTC/' in link:
                    download_links.append(link)
        
        # In danh sách các đường link để kiểm tra
        print("Danh sách các đường link Download:")
        for link in download_links:
            print(link)

    except Exception as e:
        print(f"Lỗi: {e}")
    finally:
        # Đóng trình duyệt sau khi hoàn thành
        driver.quit()
    
    # Trả về danh sách các đường link
    return download_links


In [18]:
# Gọi hàm với URL mong muốn
url = "https://cafef.vn/du-lieu/cong-bo-thong-tin.chn"  # Thay bằng URL thực tế của trang web
links = getURL(url)

Danh sách các đường link Download:
https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/00002469135802469136ctcp-bcg-land03022025-000000bo-co-ti-chnh-qu-4-2024-ca-cng-ty-m.pdf
https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/0012345679012345678ctcp-xy-dng-coteccons03032025-000000bo-co-ti-chnh-m-bn-nin.pdf
https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/0006913580246913581ctcp-xy-dng-coteccons24012025-000000bo-co-ti-chnh-hp-nht-qu.pdf
https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/0012345679012345678ctcp-xy-dng-coteccons03032025-000000bo-co-ti-chnh-hp-nht-bn-nin.pdf
https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/0006913580246913581ctcp-xy-dng-coteccons24012025-000000bo-co-ti-chnh-m-qu.pdf
https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/0007407407407407408ctcp-a-c-first-real17012025-000000bo-co-ti-chnh-hp-nht-qu.pdf
https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/HSG_Baocaotaichinh

In [19]:
len(links)

20

In [30]:
import json

In [31]:
def getURL(ori_url, num_pages, output_file="download_links.json"):
    """
    Hàm crawl link tải xuống từ các trang web.
    - ori_url: URL của trang web cần crawl.
    - num_pages: Số lượng trang muốn crawl (truyền vào khi gọi hàm).
    - output_file: Tên file để lưu danh sách link (mặc định là 'download_links.json').
    """
    # Đường dẫn đến msedgedriver
    driver_path = "C:/WebDriver/msedgedriver.exe"  # Thay đổi đường dẫn theo nơi bạn đặt msedgedriver
    
    # Cấu hình dịch vụ cho Microsoft Edge
    service = Service(executable_path=driver_path)
    
    # Khởi tạo driver cho Microsoft Edge
    driver = webdriver.Edge(service=service)
    
    # Danh sách chứa tất cả các danh sách link (list1, list2, ..., listN)
    all_download_lists = []
    
    try:
        # Mở đường dẫn
        driver.get(ori_url)
        time.sleep(2)  # Đợi trang tải xong

        # Tìm phần tử cần click bằng XPath (các bước click trước đó của bạn)
        element_to_click1 = driver.find_element(By.XPATH, '//*[@id="pagewrap"]/div[1]/div[3]/div[1]/div[1]/div[2]/div[2]/div/label/i')
        element_to_click1.click()
        time.sleep(2)
        
        element_to_click2 = driver.find_element(By.XPATH, '//*[@id="pagewrap"]/div[1]/div[3]/div[1]/div[1]/div[2]/div[2]/div/div/div[5]')
        element_to_click2.click()
        
        element_to_click2 = driver.find_element(By.XPATH, '//*[@id="pagewrap"]/div[1]/div[3]/div[1]/div[1]/div[2]/div[2]/div/div/div[6]')
        element_to_click2.click()

        # Đợi một chút để trang tải hoàn toàn sau khi click
        time.sleep(5)

        # Lặp qua số trang được truyền vào (num_pages)
        for page in range(num_pages):  # Từ trang 1 đến trang num_pages
            # Danh sách để lưu các đường link của trang hiện tại
            download_links = []
            
            try:
                # Tìm tất cả các phần tử Download trong cột "Tài về"
                download_elements = driver.find_elements(By.XPATH, '//table//tr/td[last()]/a')
                
                # Lấy thuộc tính href từ từng phần tử và lọc các link
                for element in download_elements:
                    link = element.get_attribute('href')
                    if link:  # Kiểm tra nếu link không rỗng
                        # Chỉ thêm link nếu nó chứa đuôi .pdf hoặc chứa đường dẫn của file báo cáo
                        if link.endswith('.pdf') or '/BCTC/' in link:
                            download_links.append(link)
                
                # Thêm danh sách link của trang hiện tại vào all_download_lists
                all_download_lists.append(download_links)
                
                # In danh sách các đường link của trang hiện tại để kiểm tra
                # print(f"Danh sách các đường link Download (Trang {page + 1}):")
                # for link in download_links:
                #     print(link)
                # print("-" * 50)

                # Kiểm tra xem có nút "Next" để chuyển trang không
                try:
                    next_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, '//*[@id="paging-right"]'))
                    )
                    # Cuộn đến nút Next để đảm bảo nó có thể được click
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    next_button.click()
                    
                    # Đợi vài giây để trang tiếp theo tải xong
                    time.sleep(5)
                
                except Exception as e:
                    print(f"Không thể chuyển sang trang tiếp theo (Trang {page + 1}): {e}")
                    break  # Thoát vòng lặp nếu không còn trang tiếp theo

            except Exception as e:
                print(f"Lỗi khi lấy link ở trang {page + 1}: {e}")
                break

    except Exception as e:
        print(f"Lỗi: {e}")
    finally:
        # Lưu tất cả các danh sách link vào file JSON
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(all_download_lists, f, ensure_ascii=False, indent=4)
            print(f"Đã lưu tất cả các danh sách link vào file: {output_file}")
        except Exception as e:
            print(f"Lỗi khi lưu file: {e}")
        
        # Đóng trình duyệt sau khi hoàn thành
        driver.quit()
    
    # Trả về danh sách chứa tất cả các danh sách link
    return all_download_lists

In [32]:
# Gọi hàm với URL mong muốn
url = "https://cafef.vn/du-lieu/cong-bo-thong-tin.chn"  # Thay bằng URL thực tế của trang web
all_lists = getURL(url, num_pages=66, output_file="links_of_first_66_page.json")

Đã lưu tất cả các danh sách link vào file: links_of_66_first_page.json


In [ ]:
# In lại tất cả các danh sách để kiểm tra
for i, link_list in enumerate(all_lists, 1):
    print(f"List{i}: {len(link_list)}")

In [ ]:
import json

# Đọc file JSON để lấy lại các danh sách link
with open('links_of_first_66_page.json', 'r', encoding='utf-8') as f:
    loaded_lists = json.load(f)

# In lại để kiểm tra
for i, link_list in enumerate(loaded_lists, 1):
    print(f"List{i}: {link_list}")

In [41]:
import requests
from urllib.parse import urlparse

def download_files_from_links(json_file_path="links_of_first_66_page.json", download_dir="Data"):
    """
    Hàm tải file trực tiếp từ các link trong file JSON và lưu vào thư mục Data.
    - json_file_path: Đường dẫn đến file JSON chứa các danh sách link.
    - download_dir: Thư mục để lưu các file tải xuống (mặc định là 'Data').
    """
    # Đọc file JSON để lấy các danh sách link
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            all_download_lists = json.load(f)
        print(f"Đã đọc file JSON: {json_file_path}")
    except Exception as e:
        print(f"Lỗi khi đọc file JSON: {e}")
        return


    # Duyệt qua từng danh sách link (list1, list2, ..., listN)
    for page_idx, link_list in enumerate(all_download_lists, 1):
        print(f"Xử lý trang {page_idx} với {len(link_list)} link...")
        
        # Duyệt qua từng link trong danh sách
        for link_idx, link in enumerate(link_list, 1):
            try:
                print(f"Tải link {link_idx}/{len(link_list)}: {link}")
                
                # Tải file từ URL
                response = requests.get(link, stream=True)
                response.raise_for_status()  # Kiểm tra lỗi HTTP

                # Lấy tên file từ URL
                parsed_url = urlparse(link)
                file_name = os.path.basename(parsed_url.path)
                if not file_name:  # Nếu không lấy được tên file, đặt tên mặc định
                    file_name = f"file_{page_idx}_{link_idx}.pdf"

                # Đường dẫn lưu file
                file_path = os.path.join(download_dir, file_name)

                # Lưu file vào thư mục Data
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                print(f"Đã tải và lưu file: {file_path}")

            except Exception as e:
                print(f"Lỗi khi tải link {link}: {e}")
                continue

        print(f"Hoàn thành trang {page_idx}")
        print("-" * 50)

    print("Đã hoàn thành quá trình tải xuống. Các file được lưu trong thư mục:", download_dir)


In [ ]:
# Gọi hàm để tải file
json_file_path = "links_of_first_66_page.json"  # Đường dẫn đến file JSON
download_dir = "Data"  # Thư mục để lưu file tải xuống
download_files_from_links(json_file_path, download_dir)

Đã đọc file JSON: links_of_first_66_page.json
Xử lý trang 1 với 20 link...
Tải link 1/20: https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/00002469135802469136ctcp-bcg-land03022025-000000bo-co-ti-chnh-qu-4-2024-ca-cng-ty-m.pdf
Đã tải và lưu file: Data\00002469135802469136ctcp-bcg-land03022025-000000bo-co-ti-chnh-qu-4-2024-ca-cng-ty-m.pdf
Tải link 2/20: https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/0006913580246913581ctcp-xy-dng-coteccons24012025-000000bo-co-ti-chnh-m-qu.pdf
Đã tải và lưu file: Data\0006913580246913581ctcp-xy-dng-coteccons24012025-000000bo-co-ti-chnh-m-qu.pdf
Tải link 3/20: https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/0006913580246913581ctcp-xy-dng-coteccons24012025-000000bo-co-ti-chnh-hp-nht-qu.pdf
Đã tải và lưu file: Data\0006913580246913581ctcp-xy-dng-coteccons24012025-000000bo-co-ti-chnh-hp-nht-qu.pdf
Tải link 4/20: https://cafef1.mediacdn.vn//Images/Uploaded/DuLieuDownload/BCTC/0012345679012345678ctcp-xy-dng-cotec